In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import normaltest, shapiro, jarque_bera, anderson
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
df = pd.read_csv('../../../data/data.csv')
print(df.columns)
print(df.info())
# -----------------------------
# Features base (única fuente de verdad)
# -----------------------------
features_kmeans = [
    'Age',
    'PlayTimeHours',
    'SessionsPerWeek',
    'AvgSessionDurationMinutes',
    'InGamePurchases'
]

# -----------------------------
# 1) K-Means + PCA
# -----------------------------
X_km = df[features_kmeans].copy()

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_km)

pca = PCA(n_components=3, random_state=42)
X_pca = pca.fit_transform(X_scaled)

kmeans = KMeans(
    n_clusters=3,
    init='k-means++',
    n_init=100,
    random_state=42
)

df['Cluster_PCA'] = kmeans.fit_predict(X_pca)

# -----------------------------
# 2) One-hot del cluster
# -----------------------------
X_cluster = pd.get_dummies(df['Cluster_PCA'], prefix='cluster')

# -----------------------------
# 3) DataFrame FINAL para entrenamiento
# -----------------------------
X = pd.concat(
    [
        df[features_kmeans].reset_index(drop=True),
        X_cluster.reset_index(drop=True)
    ],
    axis=1
)

y = df['EngagementLevel'].map({'Low': 0, 'Medium': 1, 'High': 2})

# -----------------------------
# 4) Comprobaciones
# -----------------------------
print("Columnas de X:")
print(X.columns)

print("\nInfo de X:")
print(X.info())



Index(['PlayerID', 'Age', 'Gender', 'Location', 'GameGenre', 'PlayTimeHours',
       'InGamePurchases', 'GameDifficulty', 'SessionsPerWeek',
       'AvgSessionDurationMinutes', 'PlayerLevel', 'AchievementsUnlocked',
       'EngagementLevel'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40034 entries, 0 to 40033
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   PlayerID                   40034 non-null  int64  
 1   Age                        40034 non-null  int64  
 2   Gender                     40034 non-null  object 
 3   Location                   40034 non-null  object 
 4   GameGenre                  40034 non-null  object 
 5   PlayTimeHours              40034 non-null  float64
 6   InGamePurchases            40034 non-null  int64  
 7   GameDifficulty             40034 non-null  object 
 8   SessionsPerWeek            40034 non-null  int64  
 9   AvgSe

# Arbol de decisión

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# ===== 1) MODELO BASE =====
base_model = DecisionTreeClassifier(
    criterion='gini',
    max_depth=None,
    min_samples_leaf=5,
    min_samples_split=4,
    splitter='random',
    random_state=42
)

# Entrenar
base_model.fit(X_train, y_train)

# Predicciones
y_pred = base_model.predict(X_test)



# ===== 2) GRID SEARCH (OPTIMIZA F1 MACRO) =====
param_grid = {
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random']
}

grid_search = GridSearchCV(
    estimator=DecisionTreeClassifier(random_state=42),
    param_grid=param_grid,
    cv=5,
    scoring='f1_macro',   # 👈 métrica principal
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_

# Predicciones del mejor modelo
y_pred_best = best_model.predict(X_test)

# ---- Métricas (SOLO 3) ----
acc_best = accuracy_score(y_test, y_pred_best)
f1m_best = f1_score(y_test, y_pred_best, average='macro')
cm_best = confusion_matrix(y_test, y_pred_best)

print("\n=== Mejor modelo (GridSearchCV) ===")
print(f"Accuracy: {acc_best:.4f}")
print(f"F1-score (macro): {f1m_best:.4f}")
print("Matriz de confusión:")
print(cm_best)


=== Modelo base ===
Accuracy: 0.8751
F1-score (macro): 0.8714
Matriz de confusión:
[[1767  267   59]
 [ 286 3484  109]
 [  70  209 1756]]

=== Mejor modelo (GridSearchCV) ===
Accuracy: 0.8780
F1-score (macro): 0.8742
Matriz de confusión:
[[1806  227   60]
 [ 274 3481  124]
 [  71  221 1743]]
